# Advantage of LLM sentiment analysis

In [1]:
import pandas as pd
from src.gpt_prompt import get_sentiment_general, get_sentiment_general_parallel, get_risk_score, get_sentiment_conservative, get_sentiment_general_multiple, get_sentiment_conservative_parallel

## 1 Training-Free

In [2]:

df = pd.read_csv("data/FiQA.csv")
news = df["Sentence"][0]
get_sentiment_general(news,model="gpt-3.5-turbo", company="Apple",output="polar", explanation=True)[0]

You will work as a Sentiment Analysis Expert for Financial News for Apple, focusing on financial indicators such as earnings, market trends, and investor opinions. Your answer will include 2 lines. In the first line, you will answer 1 sentence to analyze why the news is good or bad for Apple. Then, in the second line, you will answer with: BEARISH, BULLISH, NEUTRAL.


'BULLISH.'

In [3]:
news = df["Sentence"][0:2]
get_sentiment_general_parallel(news,model="gpt-3.5-turbo", output="polar", explanation=False)[0]

You will work as a Sentiment Analysis Expert for Financial News , focusing on financial indicators such as earnings, market trends, and investor opinions. You will not give explanation to your answer. Then, you will answer with: BEARISH, BULLISH, NEUTRAL.
You will work as a Sentiment Analysis Expert for Financial News , focusing on financial indicators such as earnings, market trends, and investor opinions. You will not give explanation to your answer. Then, you will answer with: BEARISH, BULLISH, NEUTRAL.
 average running time: 1.62 second


('BULLISH', '')

## 2 High Accuracy

In [4]:
testing_X = []
interposition = 60
for i in df["Sentence"][::interposition]:
    testing_X.append(i)
pool_outputs = get_sentiment_general_parallel(testing_X,model="gpt-3.5-turbo-16k", output="polar", explanation=False, threads=10)

You will work as a Sentiment Analysis Expert for Financial News , focusing on financial indicators such as earnings, market trends, and investor opinions. You will not give explanation to your answer. Then, you will answer with: BEARISH, BULLISH, NEUTRAL.
You will work as a Sentiment Analysis Expert for Financial News , focusing on financial indicators such as earnings, market trends, and investor opinions. You will not give explanation to your answer. Then, you will answer with: BEARISH, BULLISH, NEUTRAL.
You will work as a Sentiment Analysis Expert for Financial News , focusing on financial indicators such as earnings, market trends, and investor opinions. You will not give explanation to your answer. Then, you will answer with: BEARISH, BULLISH, NEUTRAL.
You will work as a Sentiment Analysis Expert for Financial News , focusing on financial indicators such as earnings, market trends, and investor opinions. You will not give explanation to your answer. Then, you will answer with: BEA

In [5]:
dic_df = {"positive":1, "neutral":0, "negative":-1}
testing_Y = []
for i in df["Sentiment"][::interposition]:
    testing_Y.append(dic_df[i])
dic_gpt = {"BULLISH" : 1, "NEUTRAL" : 0, "BEARISH" : -1,}
output_gpt = [dic_gpt[output[0]] for output in pool_outputs]

In [6]:
from sklearn.metrics import confusion_matrix, accuracy_score
print("accuracy:", accuracy_score(testing_Y, output_gpt))
print("confusion matrix: \n", confusion_matrix(testing_Y, output_gpt))

accuracy: 0.7857142857142857
confusion matrix: 
 [[13  1  1]
 [ 8 41  7]
 [ 0  4 23]]


## 3 Customization

In [7]:
news = "Huawei launches new Mate 60 smartphone series in China"
get_sentiment_general(news,model="gpt-4", output="polar", explanation=False)[0]

You will work as a Sentiment Analysis Expert for Financial News , focusing on financial indicators such as earnings, market trends, and investor opinions. You will not give explanation to your answer. Then, you will answer with: BEARISH, BULLISH, NEUTRAL.


'NEUTRAL'

In [8]:
get_sentiment_general(news,model="gpt-4", company="Apple", output="polar", explanation=False)[0]

You will work as a Sentiment Analysis Expert for Financial News for Apple, focusing on financial indicators such as earnings, market trends, and investor opinions. You will not give explanation to your answer. Then, you will answer with: BEARISH, BULLISH, NEUTRAL.


'BEARISH'

In [9]:
get_sentiment_general(news,model="gpt-4", company="Apple", output="score", explanation=False)[0]

You will work as a Sentiment Analysis Expert for Financial News for Apple, focusing on financial indicators such as earnings, market trends, and investor opinions. You will not give explanation to your answer. Then, you will answer with an integer between 1 and 10, with 1 being most BEARISH, 10 being most BULLISH.


'5'

In [10]:
get_risk_score(news, model="gpt-4", company="Apple", explanation=True)[0]

'Risk Level: 75'

## 4 Interpretability

In [11]:
get_sentiment_general(news,model="gpt-4", company="Apple", output="score", explanation=True)[1]

You will work as a Sentiment Analysis Expert for Financial News for Apple, focusing on financial indicators such as earnings, market trends, and investor opinions. Your answer will include 2 lines. In the first line, you will answer 1 sentence to analyze why the news is good or bad for Apple. Then, in the second line, you will answer with an integer between 1 and 10, with 1 being most BEARISH, 10 being most BULLISH.


"The news is potentially negative for Apple as Huawei's new smartphone series could compete with Apple's products in the Chinese market. "

## 5 More than sentiment analysis

https://www.quantconnect.com/terminal/processCache?request=embedded_backtest_34904f3cd60e043654b9fb849f9f2ac3.html